## Learning SMACH


In [ ]:
get_ipython().system_raw('roslaunch locobot_control main.launch use_arm:=true use_base:=true & ')

### Creating a State Machine

To create a Smach state machine, you first create a number of states, and then add those states to a State Machine container.

In [ ]:
#!/usr/bin/env python

import rospy
import smach
from pyrobot import Robot

base_config_dict={'base_controller': 'ilqr'} 
robot = Robot('locobot', base_config=base_config_dict)

### Creating a state

In [ ]:
# define state Base_move
class Base_move(smach.State):
    def __init__(self):
        smach.State.__init__(self, outcomes=['forward','backward'])
        self.counter = 0
        forward_position = [1,0.0,0.0]
        backward_position = [0.0,0.0,0.0]

    def execute(self, userdata):
        rospy.loginfo('Executing state Base_move')
        if self.counter < 2:
            self.counter += 1
            robot.base.go_to_absolute(forward_position)
            return 'forward'
        else:
            robot.base.go_to_absolute(backward_position)
            return 'backward'

In [ ]:
# define state Arm_move
class Arm_move(smach.State):
    def __init__(self):
        smach.State.__init__(self, outcomes=['”home'])

    def execute(self, userdata):
        rospy.loginfo('Executing state Arm_move')
        robot.arm.go_home()
        return 'home'

### Adding states to a state machine

In [ ]:
# main
def main():
    rospy.init_node('smach_example_state_machine')

    # Create a SMACH state machine
    sm = smach.StateMachine(outcomes=['End'])

    # Open the container
    with sm:
        # Add states to the container
        smach.StateMachine.add('Base_move', Base_move(), transitions={'forward':'Arm_move', 'backward':'End'})
        smach.StateMachine.add('Arm_move', Arm_move(), transitions={'home':'Base_move'})

    # Execute SMACH plan
    outcome = sm.execute()

if __name__ == '__main__':
    main()

[ DEBUG ] : Adding state (FOO, <__main__.Foo object at 0x7fb50d335050>, {'outcome1': 'BAR', 'outcome2': 'outcome4'})
[ DEBUG ] : Adding state 'FOO' to the state machine.
[ DEBUG ] : State 'FOO' is missing transitions: {}
[ DEBUG ] : TRANSITIONS FOR FOO: {'outcome1': 'BAR', 'outcome2': 'outcome4'}
[ DEBUG ] : Adding state (BAR, <__main__.Bar object at 0x7fb50d3351d0>, {'outcome2': 'FOO'})
[ DEBUG ] : Adding state 'BAR' to the state machine.
[ DEBUG ] : State 'BAR' is missing transitions: {}
[ DEBUG ] : TRANSITIONS FOR BAR: {'outcome2': 'FOO'}
[  INFO ] : State machine starting in initial state 'FOO' with userdata: 
	[]
[  INFO ] : State machine transitioning 'FOO':'outcome1'-->'BAR'
[  INFO ] : State machine transitioning 'BAR':'outcome2'-->'FOO'
[  INFO ] : State machine transitioning 'FOO':'outcome1'-->'BAR'
[  INFO ] : State machine transitioning 'BAR':'outcome2'-->'FOO'
[  INFO ] : State machine transitioning 'FOO':'outcome1'-->'BAR'
[  INFO ] : State machine transitioning 'BAR':'ou